<a href="https://colab.research.google.com/github/shardul28/political-stance-detection/blob/Mscproject/getting_data_XIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [105]:
!pip install mit-news-classify


In [136]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
from mitnewsclassify import tfidf as mitx

In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [138]:
cd /content/drive/MyDrive/project_data

/content/drive/MyDrive/project_data


# Fetching Data from Drive

In [139]:
infile = open('ndtv_data.pkl','rb')
ndtv_df = pickle.load(infile)

In [140]:
ndtv_df.head()

,text,url,title,content,left 0 right 1,retweet_count,favorite_count,source_name,source
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,NaN,9,63,ndtv,0
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,NaN,5,44,ndtv,0
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1.0,2,0,ndtv,0
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0.0,5,0,ndtv,0
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,NaN,3,0,ndtv,0


In [141]:
infile2 = open('republic_data.pkl','rb')
republic_df = pickle.load(infile2)

In [142]:
republic_df.head()

,text,url,title,content,left 0 right 1,retweet_count,favorite_count,source_name,source
0,"Why did Kerala have a backlog of 7,000 COVID-1...",https://www.republicworld.com/india-news/gener...,"Why Did Kerala Have A Backlog Of 7,000 COVID-1...",in a recent update to the covid situation in k...,NaN,3.0,6.0,republictv,1
1,Kerala artist creates 24-foot-long Theyyam's f...,https://www.republicworld.com/india-news/city-...,Kerala Artist Creates 24-foot-long Theyyam's F...,a kerala-based artist created an installation ...,NaN,2.0,14.0,republictv,1
2,"PM Modi launches Indian Space Association, hig...",https://www.republicworld.com/india-news/gener...,Govt as an enabler..': PM Modi Launches Indian...,modi modi on monday launched indian space sect...,1.0,4.0,27.0,republictv,1
3,"NEET Phase 2 registration 2021 to end today, h...",https://www.republicworld.com/india-news/educa...,"NEET Phase 2 Registration 2021 To End Today, H...",neet phase 2 registration: national testing ag...,NaN,1.0,5.0,republictv,1
4,"US, UK issue advisory; warn citizens to avoid ...",https://www.republicworld.com/world-news/rest-...,"Leave immediately' | US, UK Issue Advisory; Wa...","the united states and britain on monday, 11 oc...",NaN,2.0,11.0,republictv,1


In [143]:
##CHECKING NULL VALUES
print(republic_df.isna().sum())


text               440
url                440
title              440
content            440
left 0 right 1    1339
retweet_count      440
favorite_count     440
source_name          0
source               0
dtype: int64


In [144]:
print(ndtv_df.isna().sum())


text                 0
url                  0
title                0
content              0
left 0 right 1    1042
retweet_count        0
favorite_count       0
source_name          0
source               0
dtype: int64


In [145]:
republic_df=republic_df.dropna(subset=['content'])
ndtv_df=ndtv_df.dropna(subset=['content'])


In [146]:
##bias=left 0 right 1 unbiased 3

In [147]:
ndtv_df.rename(columns = {'left 0 right 1':'bias'}, inplace = True)
ndtv_df=ndtv_df.fillna({'bias':3})
ndtv_df['bias']=ndtv_df['bias'].astype(int)

In [148]:
republic_df.columns

Index(['text', 'url', 'title', 'content', 'left 0 right 1', 'retweet_count',
       'favorite_count', 'source_name', 'source'],
      dtype='object')

In [149]:
republic_df.rename(columns = {'left 0 right 1':'bias'}, inplace = True)
republic_df=republic_df.fillna({'bias':3})
republic_df['bias']=republic_df['bias'].astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [150]:
print(republic_df.isna().sum())


text              0
url               0
title             0
content           0
bias              0
retweet_count     0
favorite_count    0
source_name       0
source            0
dtype: int64


In [151]:
print(ndtv_df.isna().sum())


text              0
url               0
title             0
content           0
bias              0
retweet_count     0
favorite_count    0
source_name       0
source            0
dtype: int64


#MIT News Classify

In [185]:
# MIT News Classify is a package containing several natural language processing (NLP) and ML-based models that have been finetuned on an Annotated Corpus and its predefined news tags.

In [186]:
ndtv_df['content'][0]

"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [187]:
mitx.gettags(ndtv_df['content'][0])

['terrorism', 'armament, defense and military forces']

In [188]:
ndtv_df['content'][1]

'aryan khan did not get bail today for the third time since his arrest a week ago in the drugs-on-cruise case. his bail request will be heard on wednesday after the narcotics control bureau (ncb) gives its statement to the court. aryan khan, the 23-year-old son of actor shah rukh khan, has been in a mumbai jail since friday. he and his friend arbazz merchant were was among nine arrested after anti-drugs officials raided a cruise rave party in disguise. the anti-drugs agency had asked the court to extend his custody till friday for more time to file its reply. the court ordered the ncb to do so by wednesday morning instead, posting aryan khan\'s bail plea hearing to wednesday instead. arguing aryan khan\'s case, lawyer amit desai had requested that his request for bail be heard today. no drugs were found on him, the lawyer said. "the offence involves only a one-year sentence. for this child, seven days is not suitable," mr desai had said. but the ncb insisted it was not ready with its s

In [189]:
mitx.gettags(ndtv_df['content'][1])


['drug abuse and traffic']

In [190]:
republic_df['content'][0]


'in a recent update to the covid situation in kerala, the cpm-led kerala government on saturday, october 9, admitted on the floor of the state assembly of underreporting over 7,000 virus-related deaths in the total tally. the chairperson of the national immunisation technical advisory group (ntagi), dr nk arora, highlighting the reason for \'under-reporting\' of covid deaths, explains the process of uploading the data to the national portal. he said that kerala is not the only state where the backlog of deaths was reported but there are many states that have been late in updating the information. according to national centre for disease control (ncdc), there is great difficulty in getting accurate information on hospitalisation which is very important as it helps to understand the health system leads. however, most of the states have incomplete data concerned with hospitalisation. he further urged ncdc to work with the state to resolve the issue. further applauding the kerala governmen

In [191]:
mitx.gettags(republic_df['content'][0])

['medicine and health',
 'diseases and conditions',
 'vaccination and immunization']

In [192]:
republic_df['content'][1]

'a kerala-based artist created an installation using biscuits and other bakery products to reflect kerala\'s ceremonial art genre theyyam. suresh, also known as \'davinci,\' spent nearly 15 hours creating the massive 24-foot long art installation. the art installation was created in the hall of a kannur bakery using numerous tables arranged in the centre of the room. the face of theyyam was created using up to 25,000 biscuits of various colours and sizes, as well as other bakery products. suresh was invited to make the art piece in chef muhammad rashid\'s bakery, \'bake story.\' "we have invited davinci suresh, who made the artwork of north malabar\'s traditional art form theyyam using bakery items. bake story bakery from kannur also became part of davinci suresh\'s 100 medium plan," remarked the chef. after that, all of the bakery products were given to a veterinary farm to biodegrade. similarly, on thursday, october 7, the odisha lalit kala akademi, in collaboration with the koraput 

In [193]:
mitx.gettags(republic_df['content'][1])

['art', 'food', 'bakeries and baked products']

In [194]:
len(republic_df)

2196

In [195]:
mitx.gettags(republic_df['content'][0])


['medicine and health',
 'diseases and conditions',
 'vaccination and immunization']

In [196]:
# ##the MIT PACKAGE IDENTIFIES TAGS AS BOTH THE ORIGINAL TAG AND WITH UNITED TAGS
# for ex.
# ['united states politics and government',
#  'politics and government',
#  'united states international relations',
#  'international relations']


#  this can be just 
#  ['politics and government',
#  'international relations']

In [197]:
listx=[]
for x1 in range(len(republic_df)):
  templist=[]
  test=mitx.gettags(republic_df['content'][x1])
  for temp in test:
    if temp.find("united states") != -1:
      continue
    else:
      templist.append(temp)
  listx.append(templist)

In [198]:
republic_df['tags']=listx

In [199]:
listx=[]
for x1 in range(len(ndtv_df)):
  templist=[]
  test=mitx.gettags(ndtv_df['content'][x1])
  for temp in test:
    if temp.find("united states") != -1:
      continue
    else:
      templist.append(temp)
  listx.append(templist)

In [200]:
ndtv_df['tags']=listx


In [201]:
ndf=ndtv_df
rdf=republic_df

In [202]:
ndf.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9,63,ndtv,0,"[terrorism, armament, defense and military for..."
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5,44,ndtv,0,[drug abuse and traffic]
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2,0,ndtv,0,[appointments and executive changes]
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5,0,ndtv,0,"[suspensions, dismissals and resignations, arm..."
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3,0,ndtv,0,"[women, children and youth]"


# Sentiment Analysis

In [204]:
# VADER works better with things like slang, emojis, etc — whereas TextBlob performs strongly with more formal language usage.Therefore we will use TextBlob.
from textblob import TextBlob


0.31875

In [207]:
(ndf['content'][0])

"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [208]:
TextBlob(ndf['content'][0]).sentiment.subjectivity

0.31875

In [206]:
TextBlob(ndf['content'][0]).sentiment.polarity

-0.03125

Polarity

In [210]:
listx=[]
for x1 in range(len(ndf)):
  listx.append(TextBlob(ndf['content'][x1]).sentiment.polarity)
ndf['polarity']=listx


In [211]:
listx=[]
for x1 in range(len(rdf)):
  listx.append(TextBlob(rdf['content'][x1]).sentiment.polarity)
rdf['polarity']=listx


In [212]:
def getAnalysis(score):
  if score < 0:
    return "Negative"
  elif score == 0:
    return "Neutral"
  else:
    return "Positive"

In [213]:
rdf["polarity_score"] = rdf["polarity"].apply(getAnalysis)
ndf["polarity_score"] = ndf["polarity"].apply(getAnalysis)


In [215]:
rdf.head(5)

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score
0,"Why did Kerala have a backlog of 7,000 COVID-1...",https://www.republicworld.com/india-news/gener...,"Why Did Kerala Have A Backlog Of 7,000 COVID-1...",in a recent update to the covid situation in k...,3,3.0,6.0,republictv,1,"[medicine and health, diseases and conditions,...",0.104411,Positive
1,Kerala artist creates 24-foot-long Theyyam's f...,https://www.republicworld.com/india-news/city-...,Kerala Artist Creates 24-foot-long Theyyam's F...,a kerala-based artist created an installation ...,3,2.0,14.0,republictv,1,"[art, food, bakeries and baked products]",-0.018981,Negative
2,"PM Modi launches Indian Space Association, hig...",https://www.republicworld.com/india-news/gener...,Govt as an enabler..': PM Modi Launches Indian...,modi modi on monday launched indian space sect...,1,4.0,27.0,republictv,1,[space],0.130303,Positive
3,"NEET Phase 2 registration 2021 to end today, h...",https://www.republicworld.com/india-news/educa...,"NEET Phase 2 Registration 2021 To End Today, H...",neet phase 2 registration: national testing ag...,3,1.0,5.0,republictv,1,"[politics and government, elections, tests and...",0.111111,Positive
4,"US, UK issue advisory; warn citizens to avoid ...",https://www.republicworld.com/world-news/rest-...,"Leave immediately' | US, UK Issue Advisory; Wa...","the united states and britain on monday, 11 oc...",3,2.0,11.0,republictv,1,"[terrorism, religion and churches, bombs and e...",-0.009491,Negative


In [214]:
ndf.head(5)


,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9,63,ndtv,0,"[terrorism, armament, defense and military for...",-0.031250,Negative
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5,44,ndtv,0,[drug abuse and traffic],-0.018687,Negative
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2,0,ndtv,0,[appointments and executive changes],0.084060,Positive
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5,0,ndtv,0,"[suspensions, dismissals and resignations, arm...",0.008113,Positive
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3,0,ndtv,0,"[women, children and youth]",0.102905,Positive


Subjectivity

In [217]:
listx=[]
for x1 in range(len(ndf)):
  listx.append(TextBlob(ndf['content'][x1]).sentiment.subjectivity)
ndf['subjectivity']=listx


In [218]:
listx=[]
for x1 in range(len(rdf)):
  listx.append(TextBlob(rdf['content'][x1]).sentiment.subjectivity)
rdf['subjectivity']=listx


In [219]:
ndf.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score,subjectivity
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9,63,ndtv,0,"[terrorism, armament, defense and military for...",-0.031250,Negative,0.318750
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5,44,ndtv,0,[drug abuse and traffic],-0.018687,Negative,0.407323
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2,0,ndtv,0,[appointments and executive changes],0.084060,Positive,0.199474
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5,0,ndtv,0,"[suspensions, dismissals and resignations, arm...",0.008113,Positive,0.385273
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3,0,ndtv,0,"[women, children and youth]",0.102905,Positive,0.209667


In [220]:
rdf.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score,subjectivity
0,"Why did Kerala have a backlog of 7,000 COVID-1...",https://www.republicworld.com/india-news/gener...,"Why Did Kerala Have A Backlog Of 7,000 COVID-1...",in a recent update to the covid situation in k...,3,3.0,6.0,republictv,1,"[medicine and health, diseases and conditions,...",0.104411,Positive,0.477418
1,Kerala artist creates 24-foot-long Theyyam's f...,https://www.republicworld.com/india-news/city-...,Kerala Artist Creates 24-foot-long Theyyam's F...,a kerala-based artist created an installation ...,3,2.0,14.0,republictv,1,"[art, food, bakeries and baked products]",-0.018981,Negative,0.414815
2,"PM Modi launches Indian Space Association, hig...",https://www.republicworld.com/india-news/gener...,Govt as an enabler..': PM Modi Launches Indian...,modi modi on monday launched indian space sect...,1,4.0,27.0,republictv,1,[space],0.130303,Positive,0.530038
3,"NEET Phase 2 registration 2021 to end today, h...",https://www.republicworld.com/india-news/educa...,"NEET Phase 2 Registration 2021 To End Today, H...",neet phase 2 registration: national testing ag...,3,1.0,5.0,republictv,1,"[politics and government, elections, tests and...",0.111111,Positive,0.728858
4,"US, UK issue advisory; warn citizens to avoid ...",https://www.republicworld.com/world-news/rest-...,"Leave immediately' | US, UK Issue Advisory; Wa...","the united states and britain on monday, 11 oc...",3,2.0,11.0,republictv,1,"[terrorism, religion and churches, bombs and e...",-0.009491,Negative,0.392130


In [231]:
def getsubAnalysis(score):
  if score > 0.5:
    return "High subjectivity"
  else:
    return "Low subjectivity"

In [232]:
rdf["subjectivity_score"] = rdf["subjectivity"].apply(getsubAnalysis)
ndf["subjectivity_score"] = ndf["subjectivity"].apply(getsubAnalysis)


In [233]:
rdf.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score,subjectivity,subjectivity_score
0,"Why did Kerala have a backlog of 7,000 COVID-1...",https://www.republicworld.com/india-news/gener...,"Why Did Kerala Have A Backlog Of 7,000 COVID-1...",in a recent update to the covid situation in k...,3,3.0,6.0,republictv,1,"[medicine and health, diseases and conditions,...",0.104411,Positive,0.477418,Low subjectivity
1,Kerala artist creates 24-foot-long Theyyam's f...,https://www.republicworld.com/india-news/city-...,Kerala Artist Creates 24-foot-long Theyyam's F...,a kerala-based artist created an installation ...,3,2.0,14.0,republictv,1,"[art, food, bakeries and baked products]",-0.018981,Negative,0.414815,Low subjectivity
2,"PM Modi launches Indian Space Association, hig...",https://www.republicworld.com/india-news/gener...,Govt as an enabler..': PM Modi Launches Indian...,modi modi on monday launched indian space sect...,1,4.0,27.0,republictv,1,[space],0.130303,Positive,0.530038,High subjectivity
3,"NEET Phase 2 registration 2021 to end today, h...",https://www.republicworld.com/india-news/educa...,"NEET Phase 2 Registration 2021 To End Today, H...",neet phase 2 registration: national testing ag...,3,1.0,5.0,republictv,1,"[politics and government, elections, tests and...",0.111111,Positive,0.728858,High subjectivity
4,"US, UK issue advisory; warn citizens to avoid ...",https://www.republicworld.com/world-news/rest-...,"Leave immediately' | US, UK Issue Advisory; Wa...","the united states and britain on monday, 11 oc...",3,2.0,11.0,republictv,1,"[terrorism, religion and churches, bombs and e...",-0.009491,Negative,0.392130,Low subjectivity


In [234]:
ndf.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,tags,polarity,polarity_score,subjectivity,subjectivity_score
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9,63,ndtv,0,"[terrorism, armament, defense and military for...",-0.031250,Negative,0.318750,Low subjectivity
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5,44,ndtv,0,[drug abuse and traffic],-0.018687,Negative,0.407323,Low subjectivity
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2,0,ndtv,0,[appointments and executive changes],0.084060,Positive,0.199474,Low subjectivity
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5,0,ndtv,0,"[suspensions, dismissals and resignations, arm...",0.008113,Positive,0.385273,Low subjectivity
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3,0,ndtv,0,"[women, children and youth]",0.102905,Positive,0.209667,Low subjectivity


In [235]:
with open('/content/drive/MyDrive/project_data/ndtv_data_with_classification_and_sentiment.pkl', 'wb') as fid:
     pickle.dump(ndf, fid)
with open('/content/drive/MyDrive/project_data/republic_data_with_classification_and_sentiment.pkl', 'wb') as fid:
     pickle.dump(rdf, fid)